In [1]:
!nvidia-smi

Sat May 29 10:52:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [4]:
!mkdir -p ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json 

In [5]:
!kaggle datasets download -d alxmamaev/flowers-recognition

100% 450M/450M [00:03<00:00, 157MB/s]



In [6]:
!unzip -q flowers-recognition.zip -d /content

In [9]:
!rm -rf /content/flowers/flowers

In [10]:
#import libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os



In [11]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [12]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32
batch_size = 32

In [13]:
!pip install split_folders

In [14]:
import splitfolders

In [15]:
input_folder = '/content/flowers'
output_folder = '/content/processed_data'

splitfolders.ratio(input_folder, output_folder, seed=42, ratio=(.8, .1, .1))

Copying files: 4326 files [00:00, 4937.26 files/s]


In [16]:
img_H = 224
img_W = 224

In [17]:
# DATALOADER - 02
train_dir = '/content/processed_data/train'
test_dir = '/content/processed_data/test'
val_dir = '/content/processed_data/val'

In [18]:
# data generators
train_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                    rotation_range = 20,
                                    shear_range=0.15,
                                    zoom_range=0.15,
                                    horizontal_flip=True,
                                    fill_mode="nearest")

In [19]:
train_gen = train_data_gen.flow_from_directory(
    train_dir,
    target_size=(img_H, img_W),
    batch_size =batch_size,
    class_mode='categorical',
    subset='training')

valid_gen = train_data_gen.flow_from_directory(
    val_dir,
    target_size=(img_H, img_H),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

test_gen = train_data_gen.flow_from_directory(
    test_dir,
    target_size=(img_H, img_W),
    batch_size=1,
    class_mode='categorical',
    subset='validation')



Found 3457 images belonging to 5 classes.
Found 0 images belonging to 5 classes.
Found 0 images belonging to 5 classes.


In [20]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

9412608/9406464 [==============================] - 0s 0us/step


In [24]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(5, activation="softmax")(headModel)


In [25]:
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)


In [26]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [27]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
tf.debugging.set_log_device_placement(True)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


In [28]:
history = model.fit(train_gen, epochs=50)

Epoch 1/50
109/109 [==============================] - 70s 364ms/step - loss: 1.3471 - accuracy: 0.4608
Epoch 2/50
109/109 [==============================] - 40s 371ms/step - loss: 0.8640 - accuracy: 0.6876
Epoch 3/50
109/109 [==============================] - 41s 380ms/step - loss: 0.6910 - accuracy: 0.7512
Epoch 4/50
109/109 [==============================] - 40s 364ms/step - loss: 0.6097 - accuracy: 0.7778
Epoch 5/50
109/109 [==============================] - 39s 362ms/step - loss: 0.5566 - accuracy: 0.8021
Epoch 6/50
109/109 [==============================] - 39s 360ms/step - loss: 0.5186 - accuracy: 0.8183
Epoch 7/50
109/109 [==============================] - 39s 358ms/step - loss: 0.4675 - accuracy: 0.8360
Epoch 8/50
109/109 [==============================] - 39s 361ms/step - loss: 0.4446 - accuracy: 0.8444
Epoch 9/50
109/109 [==============================] - 39s 360ms/step - loss: 0.4446 - accuracy: 0.8415
Epoch 10/50
109/109 [==============================] - 39s 359ms/step - l

In [29]:
model.save('/content/mnv2_3c.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
model.summary() #breifs the architecture/layers/network-info

# Testing predictions

In [30]:
# make a prediction for a new image.
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(224, 224))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 224, 224, 3)
	# prepare pixel data
	img = img.astype('float32')
	img = img / 255.0
	return img

# load an image and predict the class
# load the image
img = load_image('/content/rose.jpg')
img2 = load_image('/content/sunf.jpg')
img3 = load_image('/content/tulip.jpg')
img4 = load_image('/content/dandelion.jpg')
img5 = load_image('/content/daisy.jpg')
# load model
model = load_model('/content/mnv2_3c.h5')
#result = model.predict(img)
  # predict the class
#result = model.predict_classes(img)
result = np.argmax(model.predict(img), axis=-1)
result2 = np.argmax(model.predict(img2), axis=-1)
result3 = np.argmax(model.predict(img3), axis=-1)
result4 = np.argmax(model.predict(img4), axis=-1)
result5 = np.argmax(model.predict(img5), axis=-1)
print("The image is of ",result[0])
print("The image is of ",result2[0])
print("The image is of ",result3[0])	
print("The image is of ",result4[0])
print("The image is of ",result5[0])



The image is of  2
The image is of  3
The image is of  4
The image is of  1
The image is of  0
